### 画一些论文里用到的图

#### 1. pipeline: 画一下函数拟合的两步

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.rcParams.update({
    'font.family': 'serif',  # 使用衬线字体
    'font.serif': ['Times New Roman'],  # 指定 Times New Roman 字体
    'font.size': 12,
    'text.usetex': False,
    'axes.labelsize': 13,
    'axes.titlesize': 15,
    'xtick.labelsize': 13,
    'ytick.labelsize': 13,
    'legend.fontsize': 11,
    'legend.title_fontsize': 12,
    'savefig.bbox': 'tight',
})

sns.set_style("whitegrid")
flierprops = dict(marker='o', markersize=2, linestyle='none')

In [ ]:
size_map = {
    "2160P": "3840x2160",
    "1080P": "1920x1080",
    "720P": "1280x720",
    "540P": "960x540",
    "360P": "640x360",
}
fig_dir = "/home/zhaoy/asset-fastCAE/results/vvenc/figs"
table_dir = "/home/zhaoy/asset-fastCAE/results/vvenc/tables"

In [ ]:
import pandas as pd

enc_df = pd.read_csv(f"{table_dir}/encInfo.csv")
display(enc_df)

In [ ]:
plt.figure(figsize=(5,4))
sns.lineplot(data=enc_df[(enc_df["preset"]=="faster") & (enc_df["size"] == "360P")], x="qp", y="bitrate", hue="preset", style="size", marker="o", errorbar=("ci", 0))